In [0]:
import datetime as dt

In [0]:
spark.sql("Use prod")

Out[2]: DataFrame[]

In [0]:
pcr_master = spark.sql("Select * from pcr_master")

In [0]:
pcr_master.count()

Out[4]: 2256090

In [0]:
pcr_master = spark.sql("Select * from pcr_master")

write_path = "/mnt/edfr/et3/prod/archive/pcr_master"

archive_date = dt.datetime.today().date().strftime("%m_%d_%Y")
pcr_file_archive_name = archive_date + "_PCRmaster.csv"

print(f"PCR Archive file name: {pcr_file_archive_name}")



pcr_master.write.format("delta").mode("overwrite").save(f"{write_path}/{pcr_file_archive_name}")


PCR Archive file name: 10_04_2021_PCRmaster.csv

In [0]:
mp_id_table = spark.sql("Select * from mp_id")

mp_id_file_archive_name = archive_date + "_mpid.csv"

print(f"MPID Archive file name: {mp_id_file_archive_name}")



mp_id_table.write.format("delta").mode("overwrite").save(f"{write_path}/{mp_id_file_archive_name}")


MPID Archive file name: 10_04_2021_mpid.csv

In [0]:
delinked_table = spark.sql("Select * from delinked_pairs")

delinked_file_archive_name = archive_date + "_delinked_pairs.csv"

print(f"Delinked Pairs Archive file name: {delinked_file_archive_name}")



delinked_table.write.format("delta").mode("overwrite").save(f"{write_path}/{delinked_file_archive_name}")


Delinked Pairs Archive file name: 10_04_2021_delinked_pairs.csv

In [0]:
%sql

Truncate table pcr_master;

In [0]:
%sql

Truncate table mp_id;

In [0]:
%sql

Truncate table delinked_pairs;

In [0]:
%sql

select count(1) from pcr_master;

In [0]:
%sql
select count(1) from mp_id;

In [0]:
%sql
select count(1) from delinked_pairs;

In [0]:
# Read in archived files
archive_file_path =  "/mnt/edfr/et3/prod/archive/pcr_master/"

pcr_file = "10_04_2021_PCRmaster.csv"

pcr_master = spark.read.format("delta").load(f"{archive_file_path}{pcr_file}")

pcr_master.createOrReplaceTempView("PCR_master_restore")


spark.sql('''
  MERGE INTO PCR_master M USING PCR_master_restore C
  ON M.patient_id == C.patient_id
  WHEN MATCHED 
    THEN UPDATE SET *
  WHEN NOT MATCHED
    THEN INSERT *
 ''')

In [0]:
10_04# Read in archived files
archive_file_path =  "/mnt/edfr/et3/prod/archive/pcr_master/"
mp_id_file = "10_04_2021_mpid.csv"


mp_id = spark.read.format("delta").load(f"{archive_file_path}{mp_id_file}")

mp_id.createOrReplaceTempView("mp_id_restore")

spark.sql("""
  MERGE INTO mp_id M USING mp_id_restore C
  ON M.patient_id == C.patient_id
  WHEN MATCHED 
    THEN UPDATE SET *
  WHEN NOT MATCHED 
    THEN INSERT *
""")

In [0]:
# Read in archived files
archive_file_path =  "/mnt/edfr/et3/prod/archive/pcr_master/"
delinked_pairs_file = "10_04_2021_delinked_pairs.csv"


delinked_pairs = spark.read.format("delta").load(f"{archive_file_path}{delinked_pairs_file}")

delinked_pairs.createOrReplaceTempView("delinked_restore")

spark.sql("""
  MERGE INTO delinked_pairs M USING delinked_restore C
  ON M.flagged_id == C.flagged_id
  WHEN MATCHED 
    THEN UPDATE SET *
  WHEN NOT MATCHED 
    THEN INSERT *
""")

In [0]:
%sql

select count(1) from pcr_master;

In [0]:
%sql

select count(1) from mp_id;

In [0]:
%sql

select count(1) from delinked_pairs;